# Uraniborg

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

from d2e2f.pipelines.trip_statistics import clean_statistics
from scipy.stats import norm
from myst_nb import glue
plt.style.use('presentation')
from IPython.display import display, Markdown, Latex

In [ ]:
%reload_kedro
df_stat = catalog.load('uraniborg.trip_statistics_clean')
#df_stat = catalog.load('uraniborg.trip_statistics')
df_stat['start_time'] = pd.to_datetime(df_stat['start_time'], utc=True)
df_stat['end_time'] = pd.to_datetime(df_stat['end_time'], utc=True);
#df_stat.describe()

In [ ]:
#df_stat.info()

In [ ]:
df_stat["consumption"] = df_stat["Consumption ME1 (L)"] + df_stat["Consumption ME2 (L)"]

In [ ]:
df_stat['item'] = 'raw'
df_clean = df_stat.copy()
df_clean['item'] = 'clean'
df_clean.dropna(subset=['consumption'], inplace=True)
#mask = df_clean['E'] > df_clean['E'].quantile(0.01) 
#df_clean = df_clean.loc[mask].copy()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,5)
facegrid = sns.scatterplot(data=df_clean, x='start_time',y='consumption', ax=ax);
ax.set_ylabel('(Fuel consumption per trip) [l]');

fig = ax.get_figure()
fig_name = 'overview'
glue(fig_name, fig, display=False)

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,5)
sns.histplot(df_clean, x='consumption', hue='trip_direction', ax=ax);

fig_name = 'fuel_hist'
glue(fig_name, fig, display=False)

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,5)
sns.histplot(df_clean, x='sog', hue='trip_direction', ax=ax);

fig_name = 'sog_hist'
glue(fig_name, fig, display=False)

```{glue:figure} overview
:figwidth: 1000px
:name: "overview"

Fuel consumption over time
```

```{glue:figure} fuel_hist
:figwidth: 1000px
:name: "fuel_hist"

Histogram over fuel consumption in both directions.
```

```{glue:figure} sog_hist
:figwidth: 1000px
:name: "sog_hist"

Histogram over speed over ground in both directions.
```

In [ ]:
df_ = df_clean.set_index('start_time')
df_.sort_index(inplace=True)

tänd = '2021-04-01'
utbildning = '2021-05-11'

df_.loc[:tänd,'period'] = 'släckta skärmar'
df_.loc[tänd:utbildning,'period'] = 'tända skärmar'
df_.loc[utbildning:,'period'] = 'efter utbildning'

In [ ]:
df_expanding = df_.expanding().mean()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,15)

#df_before.plot(y='consumption', label='before', ax=ax, style='b.')
#df_after.plot(y='consumption', label='after', ax=ax, style='g.')

styles = ['k.','g.','b.']
for style, (period, df_period) in zip(styles,df_.groupby(by='period', sort=False)):
    df_period.plot(y='consumption', label=period, ax=ax, style=style)


df_expanding.loc[tänd:].plot(y='consumption', label=f'expanding mean', ax=ax, style='k-', lw=3)
ax.set_ylabel('Fuel consumption [liters]')
ax.legend();

fig = ax.get_figure()
fig_name = 'fuel'
glue(fig_name, fig, display=False)

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,15)

styles = ['k.','g.','b.']
for style, (period, df_period) in zip(styles,df_.groupby(by='period', sort=False)):
    df_period.plot(y='w', label=period, ax=ax, style=style)

df_expanding.loc[tänd:].plot(y='w', label=f'expanding mean', ax=ax, style='k-', lw=3)
ylims = ax.get_ylim()
ax.set_ylim((0,ylims[1]))
ax.set_ylabel('Wind speed [m/s]')
ax.legend();

fig = ax.get_figure()
fig_name = 'wind'
glue(fig_name, fig, display=False)

In [ ]:
interesting = ['sog','consumption','w']
df_compare = df_.groupby(by='period', sort=False).mean()[interesting]
table = df_compare.style.format(precision=1)
glue("periods", table)

```{glue:figure} fuel
:figwidth: 1000px
:name: "fuel"

Fuel consumption over time
```

```{glue:figure} wind
:figwidth: 1000px
:name: "wind"

Wind speed over time
```

```{glue:figure} periods
:name: "periods"

Mean values for each period.
```

In [ ]:
M = np.tile(df_compare['consumption'],(len(df_compare),1))
changes = np.triu(M - M.T)
df_changes = pd.DataFrame(changes, index=df_compare.index, columns=df_compare.index)

columns = df_changes.columns[~(df_changes==0).all(axis=0)]
rows = df_changes.index[~(df_changes==0).all(axis=1)]
df_changes = df_changes.loc[rows,columns].copy()

df_changes = (df_changes/df_compare.loc['släckta skärmar']['consumption'])*100
df_changes

In [ ]:
glue('släckt_to_tänd', np.round(df_changes.loc['släckta skärmar','tända skärmar'],1))
glue('tänd_to_efter_utbildning', np.round(df_changes.loc['tända skärmar','efter utbildning'],1))
glue('släckt_to_efter_utbildning', np.round(df_changes.loc['släckta skärmar','efter utbildning'],1))

* Släckt to tänd: {glue:}`släckt_to_tänd`%
* Tänd to efter utbildning: {glue:}`tänd_to_efter_utbildning`%
* Släckt to efter utbildning: {glue:}`släckt_to_efter_utbildning`%


## Hypothesis testing

In [ ]:
from statsmodels.stats import weightstats as stests

In [ ]:
saving = 6
glue('saving', saving, display=False)

H0: fuel consumption has decresed {glue:}`saving`% from "släckta skärmar" to "efter utbildning".

In [ ]:
periods = df_.groupby(by='period', sort=False)
key='consumption' 
z1 = periods.get_group('släckta skärmar')[key]
z2 = periods.get_group('efter utbildning')[key]

In [ ]:
diff = saving/100*z1.mean()
ztest ,pvalue = stests.ztest(z1, z2, value=diff)
glue("pvalue",pvalue, display=False)

In [ ]:
alpha = 0.95
rejected = pvalue < 1-alpha
if rejected:
    display(Markdown(f"Null hypotesis is rejected since p-value: {np.round(pvalue,3)} is smaller than {np.round(1-alpha,3)} ({round(alpha*100,0)}% confidence)"))
else:
    display(Markdown(f"Null hypotesis cannot be rejected since p-value: {np.round(pvalue,3)} is larger than {np.round(1-alpha,3)} ({round(alpha*100,0)}% confidence)"))

In [ ]:
key ='sog'

bins = np.linspace(df_[key].min(), df_[key].max(), 100)

for direction, df_direction in df_.groupby(by='trip_direction', sort=False):
    grid = sns.displot(df_direction, x=key, kind='kde', hue='period', common_norm=False);
    fig = grid.figure
    fig.suptitle(direction)
    
    fig_name = f'{direction}'
    glue(fig_name, fig, display=False)

```{glue:figure} Ven-Landskrona
:figwidth: 1000px
:name: "Ven-Landskrona"

Ven-Landskrona
```

```{glue:figure} Landskrona-Ven
:figwidth: 1000px
:name: "Landskrona-Ven"

Landskrona-Ven
```